In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import layers
import keras
from model import CNN_with_mask

In [ ]:
def create_model(input_dim, seq_len, embed_space_size, embed_action_size):
    input_space = keras.Input(shape=(seq_len, input_dim[0]), name='Space_input')
    input_action = keras.Input(shape=(seq_len, input_dim[1]), name='Action_input')
    input_others = keras.Input(shape=(seq_len, input_dim[0], input_dim[2]), name='Other_input')

    input_shot = keras.Input(shape=(3, input_dim[0]))

    space_embedding = layers.Embedding(input_dim=embed_space_size, output_dim=4, mask_zero=True, name='Space_embedding')
    action_embeding = layers.Embedding(input_dim=embed_action_size, output_dim=4, mask_zero=True, name='Action_embedding')

    masking_layer = layers.Masking(mask_value=0)
    
    input_concat_embedding = layers.Concatenate(name='Input_merging')
    
    layer_cnn = CNN_with_mask(filters=4+4+input_dim[2], kernel_size=3, name='cnn')
    layer_dense = layers.Dense(units=3, activation='softmax')

    '''define forward'''
    inputs = [input_space, input_action, input_others]
    embeded_space = space_embedding(input_space)
    embeded_action = action_embeding(input_action)

    unmasked_others = tf.cast(input_others, tf.float32)
    masked_others = masking_layer(unmasked_others)

    embeded_input = input_concat_embedding([embeded_space, embeded_action, masked_others])

    cnn = layer_cnn(embeded_input)
    output_prob = layer_dense(cnn)

    model = keras.Model(inputs=inputs, outputs=output_prob, name='classifier')

    return model